# HW#2

In [1]:
import os
import re

# converts a txt file to the form of [string, int][]
def clean(path):
    res = []
    
    with open(path) as f:
        lines = f.readlines()
        
        for line in lines:
            words = line.split()
            # remove all leading and trailing nonalphabetic characters in each word and transform into lowercase
            # ex: -I'm, -> i'm
            sentence = ' '.join([re.sub(r'^[^a-zA-Z]+|[^a-zA-Z]+$','' , word).lower() for word in words[:-1]])
            label = int(words[-1])
            res.append([sentence, label])

    return res

# gives each word an unique number (starting from 0)
def encode(words):
    res = {}

    for word in words:
        if word not in res:
            res[word] = len(res)
            
    return res


def construct_matrix(sentences, word2num):
    M = len(sentences)
    N = len(word2num)
    res = [[0] * N for i in range(M)]

    for i, sentence in enumerate(sentences):
        for word in sentence.split():
            res[i][word2num[word]] += 1

    return res

In [2]:
# get all txt files under ./data except for readme.txt
FILES = [os.path.join('./data', f) for f in os.listdir('./data') if f.endswith('.txt') and f != 'readme.txt']
data = []

for file in FILES:
    data += clean(file)

print(len(data))
print(data[0])

3000
['a very very very slow-moving aimless movie about a distressed drifting young man', 0]


In [3]:
words = []

for x in data:
    for word in x[0].split():
        words.append(word)

word2num = encode(words)

print(len(word2num))
print([(word, num) for word, num in word2num.items() if num < 5])

5339
[('a', 0), ('very', 1), ('slow-moving', 2), ('aimless', 3), ('movie', 4)]


In [4]:
D = construct_matrix([x[0] for x in data], word2num)

print(len(D), len(D[0]))
print(D[0][:20])

3000 5339
[2, 3, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


# HW#3

In [5]:
num2word = {num: word for word, num in word2num.items()}
print(f"num2word's size: {len(num2word)}")
print(f'first 5 entries in num2word: {[(num, word) for num, word in num2word.items() if num < 5]}')

num2word's size: 5339
first 5 entries in num2word: [(0, 'a'), (1, 'very'), (2, 'slow-moving'), (3, 'aimless'), (4, 'movie')]


In [6]:
import numpy as np

col_sums = np.sum(D, axis=0)
print(f'first 5 columns sums in D: {col_sums[:5]}')

first 5 columns sums in D: [886 243   1   1 177]


In [7]:
import nltk

nltk.download('stopwords')
STOP_WORDS = set(nltk.corpus.stopwords.words('english'))
print(f'5 stop words: {np.array(list(STOP_WORDS))[:5]}')

5 stop words: ['o' 'is' 'his' 'theirs' 'were']


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/kennycartman/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
filtered_words = np.array([i for i in range(len(D[0])) if num2word[i] not in STOP_WORDS]) # remove stop words
print(f"number of words after removing stop words: {len(filtered_words)}")
print(f'first 5 words in filtered_words: {[num2word[w] for w in filtered_words[:5]]}')

number of words after removing stop words: 5195
first 5 words in filtered_words: ['slow-moving', 'aimless', 'movie', 'distressed', 'drifting']


In [9]:
filtered_words_sorted_by_freq = np.array([i for i in np.argsort(col_sums)[::-1] if i in filtered_words]) # descending
print(f'top 5 most frequent words (stop words excluded): {[num2word[i] for i in filtered_words_sorted_by_freq[:5]]}')

top 5 most frequent words (stop words excluded): ['good', 'great', 'movie', 'phone', 'film']


In [10]:
TOP_N = 2500
features = filtered_words_sorted_by_freq[:TOP_N]
features.sort()
print(f'show first 5 features after sorting: {features[:5]}')

show first 5 features after sorting: [ 4  8  9 11 15]


In [11]:
x = np.array(D)[:, filtered_words]
x_reduced = np.array(D)[:, features]
print(f'x.shape = {x.shape}')
print(f'x_reduced.shape = {x_reduced.shape}')

x.shape = (3000, 5195)
x_reduced.shape = (3000, 2500)


In [12]:
labels = np.array([pair[1] for pair in data])
print(f'number of labels: {len(labels)}')
print(f'first 5 labels: {labels[:5]}')

number of labels: 3000
first 5 labels: [0 0 0 0 1]


In [13]:
import math
from sklearn.model_selection import train_test_split

# 70% - 10% - 20% split (no feature selection)
(
    x_train_val_no_selection, 
    x_test_no_selection, 
    y_train_val_no_selection, 
    y_test_no_selection 
) = train_test_split(x, labels, test_size=0.2, random_state=1)
(
    x_train_no_selection, 
    x_val_no_selection, 
    y_train_no_selection, 
    y_val_no_selection
) = train_test_split(x_train_val_no_selection, y_train_val_no_selection, test_size=0.125, random_state=1)
print(f'x_train_no_selection.shape = {x_train_no_selection.shape}')
print(f'y_train_no_selection.shape = {y_train_no_selection.shape}')
print(f'x_val_no_selection.shape = {x_val_no_selection.shape}')
print(f'y_val_no_selection.shape = {y_val_no_selection.shape}')
print(f'x_test_no_selection.shape = {x_test_no_selection.shape}')
print(f'y_test_no_selection.shape = {y_test_no_selection.shape}')
print()

# 70% - 10% - 20% split (with feature selection)
(
    x_train_val, 
    x_test, 
    y_train_val, 
    y_test 
) = train_test_split(x_reduced, labels, test_size=0.2, random_state=1)
(
    x_train, 
    x_val, 
    y_train, 
    y_val 
) = train_test_split(x_train_val, y_train_val, test_size=0.125, random_state=1)
print(f'x_train.shape = {x_train.shape}')
print(f'y_train.shape = {y_train.shape}')
print(f'x_val.shape = {x_val.shape}')
print(f'y_val.shape = {y_val.shape}')
print(f'x_test.shape = {x_test.shape}')
print(f'y_test.shape = {y_test.shape}')

x_train_no_selection.shape = (2100, 5195)
y_train_no_selection.shape = (2100,)
x_val_no_selection.shape = (300, 5195)
y_val_no_selection.shape = (300,)
x_test_no_selection.shape = (600, 5195)
y_test_no_selection.shape = (600,)

x_train.shape = (2100, 2500)
y_train.shape = (2100,)
x_val.shape = (300, 2500)
y_val.shape = (300,)
x_test.shape = (600, 2500)
y_test.shape = (600,)


In [14]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import time

tree1 = DecisionTreeClassifier()

start_time = time.time()
tree1.fit(x_train_no_selection, y_train_no_selection)
build_time1 = time.time() - start_time

start_time = time.time()
pred = tree1.predict(x_val_no_selection)
classify_time1 = time.time() - start_time

print('[decision tree classifier (no feature selection)]')
print()
print(f'accuracy on validation dataset = {round(accuracy_score(y_val_no_selection, pred) * 100, 2)}%')
print(f'offline efficiency cost = {round(build_time1 * 1000)} ms')
print(f'online efficiency cost = {round(classify_time1 / len(y_val_no_selection) * 10 ** 6)} us')
print()

tree2 = DecisionTreeClassifier()

start_time = time.time()
tree2.fit(x_train, y_train)
build_time2 = time.time() - start_time

start_time = time.time()
pred = tree2.predict(x_val)
classify_time2 = time.time() - start_time

print('[decision tree classifier (with feature selection)]')
print()
print(f'accuracy on validation dataset = {round(accuracy_score(y_val, pred) * 100, 2)}%')
print(f'offline efficiency cost = {round(build_time2 * 1000)} ms')
print(f'online efficiency cost = {round(classify_time2 / len(y_val) * 10 ** 6)} us')
print()

[decision tree classifier (no feature selection)]

accuracy on validation dataset = 72.0%
offline efficiency cost = 2041 ms
online efficiency cost = 9 us

[decision tree classifier (with feature selection)]

accuracy on validation dataset = 77.0%
offline efficiency cost = 780 ms
online efficiency cost = 5 us



In [15]:
print(f'[decision tree classifier (no feature selection)] accuracy on testing dataset = {round(accuracy_score(y_test_no_selection, tree1.predict(x_test_no_selection)) * 100, 2)}%')
print(f'[decision tree classifier (with feature selection)] accuracy on testing dataset = {round(accuracy_score(y_test, tree2.predict(x_test)) * 100, 2)}%')

[decision tree classifier (no feature selection)] accuracy on testing dataset = 73.83%
[decision tree classifier (with feature selection)] accuracy on testing dataset = 75.67%


In [19]:
from sklearn.neighbors import KNeighborsClassifier

K = 11
knn1 = KNeighborsClassifier(n_neighbors=K, metric='cosine')
knn1.fit(x_train_no_selection, y_train_no_selection)
print(f'[knn classifier (scikit-learn, no feature selection)] accuracy on validation dataset = {round(accuracy_score(y_val_no_selection, knn1.predict(x_val_no_selection)) * 100, 2)}%')
knn2 = KNeighborsClassifier(n_neighbors=K, metric='cosine')
knn2.fit(x_train, y_train)
print(f'[knn classifier (scikit-learn, with feature selection)] accuracy on validation dataset = {round(accuracy_score(y_val, knn2.predict(x_val)) * 100, 2)}%')

[knn classifier (scikit-learn, no feature selection)] accuracy on validation dataset = 75.33%
[knn classifier (scikit-learn, with feature selection)] accuracy on validation dataset = 75.67%


In [ ]:
print(f'[knn classifier (scikit-learn, no feature selection)] accuracy on testing dataset = {round(accuracy_score(y_test_no_selection, knn1.predict(x_test_no_selection)) * 100, 2)}%')
print(f'[knn classifier (scikit-learn, with feature selection)] accuracy on testing dataset = {round(accuracy_score(y_test, knn2.predict(x_test)) * 100, 2)}%')

In [ ]:
class MyKNN:
    def __init__(self, n_neighbors=5):
        self.k = n_neighbors
        self.x_train = np.array([])
        self.y_train = np.array([])

    def fit(self, x_train, y_train):
        self.x_train = x_train
        self.y_train = y_train

    def predict(self, x):
        pred = np.zeros(len(x))

        for i, v in enumerate(x):
            dist_arr = np.zeros(len(self.x_train))
            
            for j, instance in enumerate(self.x_train):
                dist_arr[j] = 1 - self.get_sim(v, instance)

            top_k = self.y_train[np.argsort(dist_arr)[:self.k]]
            one_count = np.count_nonzero(top_k)
            zero_count = self.k - one_count
            pred[i] = 1 if one_count >= zero_count else 0

        return pred

    # cosine similarity
    def get_sim(self, v1, v2):
        norm1 = np.linalg.norm(v1)
        norm2 = np.linalg.norm(v2)

        if norm1 == 0 or norm2 == 0:
            return 0
            
        return v1.dot(v2) / (norm1 * norm2)

knn3 = MyKNN(n_neighbors=K)

start_time = time.time()
knn3.fit(x_train_no_selection, y_train_no_selection)
build_time3 = time.time() - start_time

start_time = time.time()
pred = knn3.predict(x_val_no_selection)
classify_time3 = time.time() - start_time

print('[knn classifier (my implementation, no feature selection)]')
print()
print(f'accuracy on validation dataset = {round(accuracy_score(y_val_no_selection, pred) * 100, 2)}%')
print(f'offline efficiency cost = {round(build_time3 * 10 ** 6)} us')
print(f'online efficiency cost = {round(classify_time3 / len(y_val_no_selection) * 1000)} ms')
print()

knn4 = MyKNN(n_neighbors=K)

start_time = time.time()
knn4.fit(x_train, y_train)
build_time4 = time.time() - start_time

start_time = time.time()
pred = knn4.predict(x_val)
classify_time4 = time.time() - start_time

print('[knn classifier (my implementation, with feature selection)]')
print()
print(f'accuracy on validation dataset = {round(accuracy_score(y_val, pred) * 100, 2)}%')
print(f'offline efficiency cost = {round(build_time4 * 10 ** 6)} us')
print(f'online efficiency cost = {round(classify_time4 / len(y_val) * 1000)} ms')
print()

In [ ]:
print(f'[knn classifier (my implementation, no feature selection)] accuracy on testing dataset = {round(accuracy_score(y_test_no_selection, knn1.predict(x_test_no_selection)) * 100, 2)}%')
print(f'[knn classifier (my implementation, with feature selection)] accuracy on testing dataset = {round(accuracy_score(y_test, knn2.predict(x_test)) * 100, 2)}%')